In [3]:
import lightgbm as lgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import lightgbm as lgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import joblib
import warnings

warnings.filterwarnings('ignore')



In [11]:
# Load the dataset
filename = "/Users/arthur/Documents/GITHUB/Rough_informer/Rough_Informer_for_High_freq_Order_Book/Sig_global_local_level_by_level_depth-2_row_10000.csv"
df = pd.read_csv(filename)

# Separate features and target
X = df.drop(columns=['y', 'time_step']).values  # Drop 'time_step' as it's metadata
y = df['y'].values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Replace train_test_split with:
split_2 = int(len(df) * 0.8)
split_1 = int(len(df) * 0.6)
X_train, X_test, X_val = X_scaled[:split_1], X_scaled[split_1:split_2],X_scaled[split_2:]
y_train, y_test,y_val = y[:split_1], y[split_1:split_2], y[split_2:]

In [21]:
import pandas as pd

# Load the original dataset
original_data = pd.read_csv('/Users/arthur/Documents/STUDY/Imperial/rough paths /salvi notebook/data.csv.gz', compression='gzip', nrows=500000)
original_y = original_data.iloc[:, -1].values  # Assuming 'y' is the last column

# Load the generated dataset
dataset = df

# Verify the match between 'y' values
mismatches = []
for index, row in dataset.iterrows():
    time_step = int(row['time_step'])  # Ensure the time_step is an integer
    test_y = row['y']

    # Check if time_step is within the bounds of the original dataset
    if time_step < len(original_y):
        original_y_value = original_y[time_step]
        if test_y != original_y_value:
            mismatches.append((index, time_step, test_y, original_y_value))
    else:
        mismatches.append((index, time_step, test_y, 'Out of bounds'))

# Display the mismatches
if mismatches:
    print(f"Found {len(mismatches)} mismatches:")
    for mismatch in mismatches[:10]:  # Display first 10 mismatches
        print(f"Index {mismatch[0]}: time_step={mismatch[1]}, test_y={mismatch[2]}, original_y={mismatch[3]}")
else:
    print("All 'y' values match correctly.")


All 'y' values match correctly.


In [12]:
print("X_train", X_train.shape)
print("X_val", X_val.shape)
print("y_train", y_train.shape)
print("y_val", y_val.shape)

X_train (6000, 840)
X_val (2000, 840)
y_train (6000,)
y_val (2000,)


In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
import warnings

warnings.filterwarnings('ignore')


In [13]:
# ===============================
# 2️⃣ Elastic Net Regression Model
# ===============================

# Initialize and train Elastic Net model
elastic_net = ElasticNet(alpha=0.3, l1_ratio=0.3, random_state=42)
elastic_net.fit(X_train, y_train)

# Make predictions
y_pred = elastic_net.predict(X_test)

# Calculate R² score
r2 = r2_score(y_test, y_pred)

# Print R² score
print(f"R² Score on Validation Set: {r2:.4f}")

R² Score on Validation Set: -0.0039


In [24]:

# ===============================
# 2️⃣ Hyperparameter Tuning with Hyperopt
# ===============================

param_space = {
    'boosting_type': hp.choice('boosting_type', ['dart']),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 0.6),
    'drop_rate': hp.uniform('drop_rate', 0.2, 0.9),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.3)),
    'max_depth': hp.choice('max_depth', [ 5, 6,7,8,9,10]),
    'min_child_samples': hp.randint('min_child_samples', 100, 400),
    'min_child_weight': hp.uniform('min_child_weight', 0.01, 0.05),
    'n_estimators': hp.choice('n_estimators', [900,1200,1500]),
    'objective': 'regression',
    'skip_drop': hp.uniform('skip_drop', 0.3, 0.7),
    'subsample': hp.uniform('subsample', 0.2, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    
}

def train_lgb_model(params, X_train, y_train, X_test, y_test, seed=42):
    """Train LightGBM and return (model, R^2, RMSE) on validation."""
    
    # Add additional parameters
    params.update({
        'random_state': seed,    
        'verbosity': -1,          # Suppresses output
        'n_jobs': -1              # Uses all CPU cores
    })
    model = lgb.LGBMRegressor(**params)
    
    
    # Correct way to pass early stopping
    model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='r2',  # Use MSE for early stopping
    callbacks=[lgb.early_stopping(stopping_rounds=30, verbose=False)])
    
    y_test_pred = model.predict(X_test)
    r2_test = r2_score(y_test, y_test_pred)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    return model, r2_test, rmse_test


def objective_func(space):
    _, r2_test, rmse_test = train_lgb_model(space, X_train, y_train, X_test, y_test)
    return {'loss': 1.0 - r2_test, 'status': STATUS_OK}

# Hyperparameter Optimization
trials = Trials()
best_params = fmin(
    fn=objective_func,
    space=param_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=trials,
    rstate=np.random.default_rng(42)
)

[1]	valid_0's l2: 0.571195                            
[2]	valid_0's l2: 0.571106                            
[3]	valid_0's l2: 0.571037                            
[4]	valid_0's l2: 0.571054                            
[5]	valid_0's l2: 0.571063                            
[6]	valid_0's l2: 0.571066                            
[7]	valid_0's l2: 0.571041                            
[8]	valid_0's l2: 0.571056                            
[9]	valid_0's l2: 0.570925                            
[10]	valid_0's l2: 0.570849                           
[11]	valid_0's l2: 0.570749                           
[12]	valid_0's l2: 0.570706                           
[13]	valid_0's l2: 0.570629                           
[14]	valid_0's l2: 0.570616                           
[15]	valid_0's l2: 0.570527                           
[16]	valid_0's l2: 0.570523                           
[17]	valid_0's l2: 0.570519                           
[18]	valid_0's l2: 0.570469                           
[19]	valid

In [27]:
# Map the indices to actual values
best_params['n_estimators'] = [600, 900, 1200, 1500][best_params['n_estimators']]




In [28]:
best_params['max_depth']=[ 5, 6,7,8,9,10][best_params['max_depth']]

In [29]:
print(best_params)

{'boosting_type': 'dart', 'colsample_bytree': 0.36258863099092664, 'drop_rate': 0.43931084602065473, 'learning_rate': 0.0010317976248721953, 'max_depth': 9, 'min_child_samples': 269, 'min_child_weight': 0.01315933119342986, 'n_estimators': 1200, 'reg_alpha': 0.059936189081400526, 'reg_lambda': 0.8609304819134347, 'skip_drop': 0.5432996648684546, 'subsample': 0.4403370838324067}


In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import lightgbm as lgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import joblib
import warnings

warnings.filterwarnings('ignore')

# ===============================
# 1️⃣ Data Loading & Preprocessing
# ===============================

# Load the dataset
filename = "/Users/arthur/Documents/GITHUB/Rough_informer/Rough_Informer_for_High_freq_Order_Book/final_dataset_depth-2_row_10_000.csv"
df = pd.read_csv(filename)

# Separate features and target
X = df.drop(columns=['y', 'time_step']).values  # Drop 'time_step' as it's metadata
y = df['y'].values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Replace train_test_split with:
split_idx = int(len(df) * 0.8)
X_train, X_val = X_scaled[:split_idx], X_scaled[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]
print(X.shape)


(10000, 1260)


In [18]:
print("X_train", X_train.shape)
print("X_val", X_val.shape)
print("y_train", y_train.shape)
print("y_val", y_val.shape)

X_train (8000, 1260)
X_val (2000, 1260)
y_train (8000,)
y_val (2000,)


In [20]:


# ===============================
# 3️⃣ Final Model Training
# ===============================



# Train the final model
final_model, final_r2, final_rmse = train_lgb_model(best_params, X_train, y_train, X_val, y_val)

# Save the model
joblib.dump(final_model, 'lgb_model_final.joblib')

# ===============================
# 4️⃣ Evaluation
# ===============================

print(f"Final R² on Validation Set: {final_r2:.4f}")
print(f"Final RMSE on Validation Set: {final_rmse:.4f}")


[1]	valid_0's l2: 0.551269
[2]	valid_0's l2: 0.551236
[3]	valid_0's l2: 0.551241
[4]	valid_0's l2: 0.551171
[5]	valid_0's l2: 0.551023
[6]	valid_0's l2: 0.551026
[7]	valid_0's l2: 0.551022
[8]	valid_0's l2: 0.550979
[9]	valid_0's l2: 0.550956
[10]	valid_0's l2: 0.550916
[11]	valid_0's l2: 0.550898
[12]	valid_0's l2: 0.550883
[13]	valid_0's l2: 0.550883
[14]	valid_0's l2: 0.550921
[15]	valid_0's l2: 0.550957
[16]	valid_0's l2: 0.550943
[17]	valid_0's l2: 0.550928
[18]	valid_0's l2: 0.550913
[19]	valid_0's l2: 0.550907
[20]	valid_0's l2: 0.5509
[21]	valid_0's l2: 0.550903
[22]	valid_0's l2: 0.550893
[23]	valid_0's l2: 0.55089
[24]	valid_0's l2: 0.550908
[25]	valid_0's l2: 0.550905
[26]	valid_0's l2: 0.550899
[27]	valid_0's l2: 0.55091
[28]	valid_0's l2: 0.55091
[29]	valid_0's l2: 0.55087
[30]	valid_0's l2: 0.550862
[31]	valid_0's l2: 0.550799
[32]	valid_0's l2: 0.550749
[33]	valid_0's l2: 0.550755
[34]	valid_0's l2: 0.550762
[35]	valid_0's l2: 0.550785
[36]	valid_0's l2: 0.550741
[37]	va